In [1]:
import time
import mysql.connector
import requests
from datetime import datetime, timedelta
import schedule

In [2]:
import os
from dotenv import load_dotenv


# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
service_key = os.getenv("SERVICE_KEY")

# 사용 예시
print(f"Service Key: {service_key}")


Service Key: 2B93BD36-A99E-413E-9582-F6428745D972


In [10]:
# MySQL 연결 설정
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT", 3306))
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

# MySQL 연결 생성 함수
def create_connection():
    return mysql.connector.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        pool_name="mypool",
        pool_size=5
    )


In [11]:
# API에서 데이터 가져오기 함수
def fetch_api_data():
    url = "http://marineweather.nmpnt.go.kr:8001/openWeatherNow.do"
    params = {
        "serviceKey": service_key,
        "resultType": "json",
        "mmaf": "101",
        "mmsi": "994401597",
        "dataType": "2"
    }

    # API 요청
    response = requests.get(url, params=params)

    # 응답 확인
    if response.status_code == 200:
        try:
            return response.json()  # JSON 데이터를 반환
        except Exception as e:
            print(f"Error parsing JSON response: {e}")
            return None
    else:
        print(f"Failed to fetch API data. Status code: {response.status_code}")
        return None


In [12]:
response = fetch_api_data()
print(response)


{'result': {'status': 'OK', 'message': '', 'recordset': [{'DATETIME': '20250115120000', 'MMAF_CODE': '101', 'MMAF_NM': '부산청', 'MMSI_CODE': '994401597', 'MMSI_NM': '부산항유도등부표(랜비)', 'WIND_DIRECT': '156', 'WIND_SPEED': '10.3', 'SURFACE_CURR_DRC': '49', 'SURFACE_CURR_SPEED': '.8', 'WAVE_DRC': '미제공', 'WAVE_HEIGTH': '미제공', 'AIR_TEMPERATURE': '.5', 'HUMIDITY': '22', 'AIR_PRESSURE': '1024', 'WATER_TEMPER': '13.6', 'SALINITY': '18.9', 'HORIZON_VISIBL': '미제공', 'TIDE_SPEED': '미제공', 'TIDE_DIRECT': '미제공', 'TIDE_TENDENCY': '미제공', 'LATITUDE': '35.0665', 'LONGITUDE': '129.131'}]}}


In [13]:
# MySQL 연결 객체 초기화
conn = None

def ensure_connection():
    """MySQL 연결 상태를 확인하고 필요시 재연결."""
    global conn
    if conn is None or not conn.is_connected():
        print("Database connection is not active. Attempting to reconnect...")
        conn = create_connection()
        print("Reconnected to the database.")


Connection object is created successfully.


In [14]:
def upsert_data(data):
    try:
        # 연결 상태 확인
        if conn is None or not conn.is_connected():
            print("Database connection is not active. Attempting to reconnect...")
            conn.reconnect(attempts=3, delay=5)
            print("Reconnected to the database.")

        # 삽입 쿼리
        insert_query = """
        INSERT INTO oceandata (
            datetime, mmaf_code, mmaf_name, mmsi_code, mmsi_name, wind_direct, wind_speed,
            surface_curr_drc, surface_curr_speed, air_temperature, humidity, air_pressure,
            water_temperature, salinity, latitude, longitude
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            wind_direct = VALUES(wind_direct),
            wind_speed = VALUES(wind_speed),
            surface_curr_drc = VALUES(surface_curr_drc),
            surface_curr_speed = VALUES(surface_curr_speed),
            air_temperature = VALUES(air_temperature),
            humidity = VALUES(humidity),
            air_pressure = VALUES(air_pressure),
            water_temperature = VALUES(water_temperature),
            salinity = VALUES(salinity);
        """

        # 데이터 파싱 및 삽입
        with conn.cursor() as cursor:
            if "result" in data and "recordset" in data["result"]:
                for entry in data["result"]["recordset"]:
                    cursor.execute(insert_query, (
                        datetime.strptime(entry["DATETIME"], "%Y%m%d%H%M%S"),
                        entry["MMAF_CODE"],
                        entry["MMAF_NM"],
                        entry["MMSI_CODE"],
                        entry["MMSI_NM"],
                        float(entry["WIND_DIRECT"]) if entry["WIND_DIRECT"] != "미제공" else None,
                        float(entry["WIND_SPEED"]) if entry["WIND_SPEED"] != "미제공" else None,
                        float(entry["SURFACE_CURR_DRC"]) if entry["SURFACE_CURR_DRC"] != "미제공" else None,
                        float(entry["SURFACE_CURR_SPEED"]) if entry["SURFACE_CURR_SPEED"] != "미제공" else None,
                        float(entry["AIR_TEMPERATURE"]) if entry["AIR_TEMPERATURE"] != "미제공" else None,
                        float(entry["HUMIDITY"]) if entry["HUMIDITY"] != "미제공" else None,
                        float(entry["AIR_PRESSURE"]) if entry["AIR_PRESSURE"] != "미제공" else None,
                        float(entry["WATER_TEMPER"]) if entry["WATER_TEMPER"] != "미제공" else None,
                        float(entry["SALINITY"]) if entry["SALINITY"] != "미제공" else None,
                        round(float(entry["LATITUDE"]), 5),
                        round(float(entry["LONGITUDE"]), 5)
                    ))
                conn.commit()
        print("Data successfully inserted or updated.")
    except mysql.connector.Error as e:
        print(f"MySQL error: {e}")
        if conn and not conn.is_connected():
            print("Reconnecting to MySQL...")
            conn.reconnect(attempts=3, delay=5)
    except Exception as e:
        print(f"Unexpected error: {e}")


In [16]:
api_data = fetch_api_data()
upsert_data(api_data)

schedule.every(10).minutes.do(lambda: upsert_data(fetch_api_data()))

try:
    print("Scheduler is running... Press Ctrl+C to stop.")
    while True:
        schedule.run_pending()
        time.sleep(1)
except KeyboardInterrupt:
    print("Scheduler stopped. Closing database connection...")
    if conn.is_connected():
        conn.close()


Unexpected error: 'NoneType' object has no attribute 'is_connected'
Scheduler is running... Press Ctrl+C to stop.
Scheduler stopped. Closing database connection...


AttributeError: 'NoneType' object has no attribute 'is_connected'